In [ ]:
# import something or python is useless...
import pandas as pd
import numpy as np

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from sklearn import neighbors
from sklearn import cross_validation
from sklearn import metrics
from sklearn.grid_search import GridSearchCV

from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import Imputer, StandardScaler,MinMaxScaler
from sklearn import preprocessing
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin

from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn import metrics
from sklearn.svm import LinearSVC, SVC, LinearSVR, SVR
from sklearn import svm

from sklearn import tree
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import neighbors
from sklearn import gaussian_process

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.neighbors import KNeighborsClassifier

#from sklearn.isotonic import IsotonicRegression
from sklearn import neural_network

from sklearn import grid_search
from sklearn import feature_extraction, feature_selection

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction import DictVectorizer

In [ ]:
import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
from sklearn.decomposition import PCA 

In [ ]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
class SelectOneColumn(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # return X.ix[:, self.column]
        return X[self.column]

In [ ]:
class DictVectSeries(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):   
        if ((type(X[X.index[0]]) is str) | (type(X[X.index[0]]) is unicode)) == False:
            X = X.astype(str)       
        samples = [dict(enumerate(sample)) for sample in X]        
        self.dict_ = DictVectorizer()      
        self.dict_.fit(samples)       
        return self

    
    def transform(self, X, y=None):      
        if ((type(X[X.index[0]]) is str) | (type(X[X.index[0]]) is unicode)) == False:
            X = X.astype(str)   
        samples = [dict(enumerate(sample)) for sample in X]      
        return self.dict_.transform(samples)
    

In [ ]:
class DenseTransformer(BaseEstimator, TransformerMixin):

    def fit_transform(self, X, y=None):
        return self.transform(X)

    def transform(self, X):
        return X.todense()

    def fit(self, X, y=None):
        return self

In [ ]:
class DFback(BaseEstimator, TransformerMixin):
    def __init__(self):
        self = self

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # return X.ix[:, self.column]
        return pd.DataFrame(X)

In [ ]:
class SeriesImputer(BaseEstimator, TransformerMixin):
    """ Impute missing values.
       Columns of dtype object are imputed with the most frequent value 
       in column.

       Columns of other types are imputed with mean of column.
    """
       # def __init__(self)
   
    def fit(self, X, y=None):
        if len(X.mode()) != 0 :
            self.fill = (X.mode()[0] if ((X.dtype == np.dtype('O')) | (X.dtype == np.dtype(tuple)))  else X.mean())
        else :      
            self.fill = X[X.first_valid_index()]        
        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [ ]:
class GetVectorizeTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self = self
        
    def fit(self, X, *_):
        return self
    
    def transform(self, X, *_):
       # if isinstance(X, pd.Series):
        return feature_extraction.text.TfidfVectorizer().fit_transform(X)
       # else:
     #       raise TypeError("This transformer only works with Pandas Dataframes")
    

In [ ]:
# read the training file to see what mess we have...
df = pd.read_csv('fullzip')
df = df.drop("Unnamed: 0", axis = 1)

In [ ]:
# look at the headers...
df.head(3)

In [ ]:
# convert date time to date time
df.timeStamp = pd.to_datetime(df.timeStamp)

In [ ]:
# check the data types and see where our blank fields are...
df.info()

In [ ]:
# so no need to reload again in case messed up
df_original = df.copy()

In [ ]:
# make time stamp the index
df.set_index('timeStamp', inplace=True)
df.head(3)

In [ ]:
df.describe()
#seems all are categorical

In [ ]:
#guys this is for checking how many unique fields for each column, maybe useful for you guys in future
[(c, df[c].value_counts().count()) for c in list(df.columns)]

In [ ]:
#maybe we should treat 'lat' and 'lng' as a pair rather than view separately?
df['coordinate'] = zip(df['lat'],df['lng'])

In [ ]:
len(df['coordinate'].unique())
#great, it's pretty much the same

In [ ]:
df.groupby('coordinate').count().sort_values('emergencytype',ascending=False).head()
#this may tell us something


In [ ]:
df[df['coordinate'] == (40.0972222, -75.3761952)]

In [ ]:
pipe_list_dummies = []

In [ ]:
pipe_list_dummies.append( ('coordinate', make_pipeline(SelectOneColumn('coordinate'),DictVectSeries(),DenseTransformer())) )

In [ ]:
pipe_list_dummies.append( ('zip', make_pipeline(SelectOneColumn('zip'),SeriesImputer(),DictVectSeries(),DenseTransformer())) )

In [ ]:
pipe_list_dummies.append( ('twp', make_pipeline(SelectOneColumn('twp'),SeriesImputer(),DictVectSeries(),DenseTransformer())) )

In [ ]:
#pipe_list_dummies.append( ('desc', make_pipeline(SelectOneColumn('desc'),SeriesImputer(),TfidfVectorizer(),DenseTransformer())) )

In [ ]:
pipe_list_dummies.append( ('addr', make_pipeline(SelectOneColumn('addr'),SeriesImputer(),TfidfVectorizer(),DenseTransformer())) )

In [ ]:
#coordinate_pipe = Pipeline([('Col_coordinate', SelectOneColumn('coordinate')), 
                      #('Fill_na_coordinate', SeriesImputer()),
                      #                      ('Dummy_coordinate', DictVectSeries())])

In [ ]:
#df.coordinate.mode

In [ ]:
union = FeatureUnion(pipe_list_dummies)

In [ ]:
union_model_pipe = Pipeline([('union', union),                     
                    ('Model', RandomForestClassifier())])

In [ ]:
cv = cross_validation.KFold(len(df), n_folds=5, shuffle=True)

In [ ]:
results = cross_validation.cross_val_score(union_model_pipe, df, df['emergencytype'], cv=cv, verbose=2)

In [ ]:
results

In [ ]:
results.mean()

In [ ]:
# Lazy mode

# What happens if we drop na?
data = df.dropna()
len(data)

In [ ]:
# 67626 non-blank record seems not bad la...

# continue lazy mode!!

```python
list(data.columns)


col = ['id', # key
       'lat', # location
       'lng', # location
       'desc', # free text
       'zip', # location
       'twp', # location
       'addr', # location
       'e', # unknown data?
       'emergencytype', # categorical
       'emergencysubtype' # categorical
       ]
 ```

In [ ]:
# look at categorical data
cat_cols = ['e', # unknown data?
            'emergencytype', # categorical
            'emergencysubtype' # categorical
           ]
for col in cat_cols:
    print col, ':', len(data[col].unique())

In [ ]:
#sleep